lunana  
last update 2022 04 28  
ゆっくりしていってね！  

# Trends
## EDA
https://www.kaggle.com/lunapandachan/h-m-eda-english

**霊夢:今日はByfone氏のnotebookを高速化するよ。  
魔理沙:元々は2時間以上かかる。**

**Reimu: Today is Byfone's speed up exam.  
Marisa: Over two hours to about a hour.**  

https://www.kaggle.com/code/byfone/h-m-trending-products-weekly

**霊夢:データが多いのでtest=Trueの時は1000個のデータを使うよ。**  

**Reimu: Since there is a lot of data, 1000 data will be used when test = True.**

In [ ]:
test=False

In [ ]:
import numpy as np
import pandas as pd

from math import sqrt
from pathlib import Path
from tqdm import tqdm
import time
import gc
tqdm.pandas()

In [ ]:
data_path = Path('../input/h-and-m-personalized-fashion-recommendations/')
N = 12

**魔理沙:N=12は予測アイテムの最大数だぜ。**

**Marisa: N = 12 is the maximum number of predictable items.**

### Read the train data
trainデータの読み込み

In [ ]:
df = pd.read_csv(data_path / 'transactions_train.csv',
                 usecols = ['t_dat', 'customer_id', 'article_id'],
                 dtype={'article_id': str})

if test :
    df=df[:1000]

df['t_dat'] = pd.to_datetime(df['t_dat'])
last_ts = df['t_dat'].max()

**霊夢:customers.csvとarticles.csvは使わないの？？  
魔理沙:imageも使わないのかよ？？**　　

**Reimu: Don't you use customers.csv and articles.csv??  
Marisa: Don't you use image too??**

### Add the last day of billing week

請求週の最終日を追加します

**霊夢:これは何でやってるのかな？
魔理沙:必要あるの？**　　

**Reimu: Why are you doing this?
Marisa: Do you need it?**

In [ ]:
import datetime
last_ts2= df['t_dat'].max()
first_ts= df['t_dat'].min()

In [ ]:
%time
df0=pd.DataFrame(index=[],columns=df.columns)
while last_ts2>first_ts:
    next_ts=last_ts2-datetime.timedelta(days=7)
    df2=df[(last_ts2 >= df['t_dat'])&(df['t_dat'] >next_ts) ].copy()
    df2['ldbw']=last_ts2
    df0=pd.concat([df0,df2])
    #print(last_ts)
    last_ts2=next_ts

In [ ]:
df0 = df0.sort_values('t_dat').reset_index(drop=True)
df0

In [ ]:
df

In [ ]:
#df['ldbw'] = df['t_dat'].progress_apply(lambda d: last_ts - (last_ts - d).floor('7D'))
df=df0
gc.collect()

In [ ]:
import pickle
df.to_pickle('df.pkl')

### Count the number of transactions per week 

週あたりのトランザクション数を数える

In [ ]:
weekly_sales = df.drop('customer_id', axis=1).groupby(['ldbw', 'article_id']).count()
weekly_sales = weekly_sales.rename(columns={'t_dat': 'count'})
weekly_sales

In [ ]:
df = df.join(weekly_sales, on=['ldbw', 'article_id'])

In [ ]:
df.head()

In [ ]:
df['count'].unique()

### Let's assume that in the target week sales will be similar to the last week of the training data

目標週の売上高がトレーニングデータの先週と同様になると仮定しましょう

In [ ]:
weekly_sales = weekly_sales.reset_index().set_index('article_id')
last_day = last_ts.strftime('%Y-%m-%d')

df = df.join(
    weekly_sales.loc[weekly_sales['ldbw']==last_day, ['count']],
    on='article_id', rsuffix="_targ")

df['count_targ'].fillna(0, inplace=True)
del weekly_sales

### Calculate sales rate adjusted for changes in product popularity 

製品の人気の変化に合わせて調整された販売率を計算します

In [ ]:
df['quotient'] = df['count_targ'] / df['count']

In [ ]:
df.head()

### Take supposedly popular products

おそらく人気のある製品を取る

In [ ]:
target_sales = df.drop('customer_id', axis=1).groupby('article_id')['quotient'].sum()
general_pred = target_sales.nlargest(N).index.tolist()
del target_sales

### Fill in purchase dictionary

購入辞書に記入する

In [ ]:
purchase_dict = {}

for i in tqdm(df.index):
    cust_id = df.at[i, 'customer_id']
    art_id = df.at[i, 'article_id']
    t_dat = df.at[i, 't_dat']

    if cust_id not in purchase_dict:
        purchase_dict[cust_id] = {}

    if art_id not in purchase_dict[cust_id]:
        purchase_dict[cust_id][art_id] = 0
    
    x = max(1, (last_ts - t_dat).days)

    a, b, c, d = 2.5e4, 1.5e5, 2e-1, 1e3
    y = a / np.sqrt(x) + b * np.exp(-c*x) - d

    value = df.at[i, 'quotient'] * max(0, y)
    purchase_dict[cust_id][art_id] += value

**魔理沙:個人の趣向に合わせて、購入履歴×トレンドのデータベースを作ったんだな。**

**Marisa: I created a database of purchase history and trends to suit my personal taste.**

### Make a submission

In [ ]:
sub = pd.read_csv(data_path / 'sample_submission.csv')

pred_list = []
for cust_id in tqdm(sub['customer_id']):
    if cust_id in purchase_dict:
        series = pd.Series(purchase_dict[cust_id])
        series = series[series > 0]
        l = series.nlargest(N).index.tolist()
        if len(l) < N:
            l = l + general_pred[:(N-len(l))]
    else:
        l = general_pred
    pred_list.append(' '.join(l))

sub['prediction'] = pred_list
sub.to_csv('submission.csv', index=None)

**霊夢:今まで購入したモノの中で流行にあったものを再購入したリストだぜ。  
魔理沙:そりゃあ２％しかないわ**

**Reimu:It's a list of repurchased items that were trendy among the items I bought so far.   
Marisa:That's only 2%**

In [ ]:
sub.head()

**霊夢：残りの９７％は売れ筋の違う柄か、同じ柄の違う型を選んでるんじゃない？
魔理沙：価格帯もあるよね。3000円の服しか買わない人は3万円の服買わないよね**

**Reimu:The remaining 97% are choosing different patterns that sell well, or different types with the same pattern, aren't they?  
Marisa:There is also a price range. People who buy only 3,000 yen clothes don't buy 30,000 yen clothes, right?**